<a href="https://colab.research.google.com/github/JuanArocaMIAR/03MIAR---Algoritmos-de-Optimizacion---2023/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Juan_Aroca_Perez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: xxxxxxx xxxxxxxxxxx xxxxxxxxxx  <br>
Url: https://github.com/LuisClauss/Algoritmos-de-optimizacion-VIU.git<br>
Google Colab: https://colab.research.google.com/drive/xxxxxxxxxxxxxxxx <br>
Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Configuración de Tribunales

Descripción del problema:(copiar enunciado)

....







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

### **Problema 1:** Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible.

In [30]:
#Respuesta
import numpy as np
import pandas as pd
from google.colab import files

# Subir el archivo CSV desde tu sistema local
archivo = files.upload()

# Seleccionar el archivo
datos = list(archivo.keys())[0]

# Cargar el dataframe desde el archivo CSV
df_actores = pd.read_csv(datos, index_col=0)


Saving Datos problema doblaje(30 tomas, 10 actores) - Hoja 1.csv to Datos problema doblaje(30 tomas, 10 actores) - Hoja 1 (1).csv


In [36]:
matriz_tomas = df_actores.iloc[1:31, 0:10].astype(int).values
print(matriz_tomas)

[[1 1 1 1 1 0 0 0 0 0]
 [0 0 1 1 1 0 0 0 0 0]
 [0 1 0 0 1 0 1 0 0 0]
 [1 1 0 0 0 0 1 1 0 0]
 [0 1 0 1 0 0 0 1 0 0]
 [1 1 0 1 1 0 0 0 0 0]
 [1 1 0 1 1 0 0 0 0 0]
 [1 1 0 0 0 1 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 0]
 [1 1 0 0 0 1 0 0 1 0]
 [1 1 1 0 1 0 0 1 0 0]
 [1 1 1 1 0 1 0 0 0 0]
 [1 0 0 1 1 0 0 0 0 0]
 [1 0 1 0 0 1 0 0 0 0]
 [1 1 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 1]
 [1 0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [1 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 0]
 [1 1 1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 1]
 [1 0 1 0 1 0 0 0 1 0]
 [0 0 0 1 1 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 1 1 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0]]


 La idea para resolver este programa es el empleo del problema tipo visto en la asignatura de **Programación lineal entera** puesto que, para empezar, todas las variables son de tipo entero. Además, cumple con la premisa de que hay que buscar un máximo o mínimo de una función objetivo lineal sujeto a restricciones, que en este caso, dichas restricciones serán que los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben y no es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

El objetivo del problema será **minimizar el gasto** total de los actores de doblaje. Por otro lado, como algoritmo de búsqueda de la solución, **vamos a emplear ramificación y poda**, para ello, necesitamos una estructura de árbol. Los actores y las tomas ya están colocados en la tabla.

 Las primeras **preguntas** que debemos hacernos son: ¿Cómo diseñamos el árbol? ¿Qué mecanismo diseñamos para ramificar? ¿Cómo podamos?.

 Una posible manera de **diseñar el árbol** es mediante una lista de tuplas, donde cada tupla representa una asignación parcial. Cada elemento de la tupla corresponde a un actor, y el valor en ese elemento indica a qué toma ha sido asignado ese actor como se vio en teoría. En el código que proporcionaste, una tupla del árbol de soluciones tiene la forma (a1, a2, ..., an), donde a_i es el número de toma a la cual el actor i ha sido asignado.

 Vamos a proveernos de una función para estimar la **cota inferior para una solución parcial**. Tenemos que hacer una estimación, esto es la que nos va a servir para podar. Se trata de una estrategia para conducir al algoritmo a que sea más eficiente, o, a que dirija la búsqueda para encontrar la solución óptima. Para la cota inferior lo que **buscamos es el mínimo de esas columnas** para que nos sirva como función de cota inferior.

#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

Dado que tenemos 10 actores y 30 tomas, tenemos un espacio de soluciones de 30**10 ya que cada actor puede ser asignado a cualquiera de las 30 tomas. El valor del espacio de soluciones es muy grande por lo que destaca la complejidad de explorar todo el espacio de soluciones y la necesidad de algoritmos como ramificación y poda para encontrar soluciones óptimas de manera eficiente.

El número total de soluciones tiene una expresión factorial. Esto se debe a que cada actor tiene 30 tomas entre las cuales puede ser asignado, y hay 10 actores en total, lo que supone un orden de complejidad O(N!), por lo que la técnica de ramificación y poda puede ser muy útil para reducir el número de búsqueda de soluciones.

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

Lo siguiente será aplicar la **técnica de ramificación y poda**. Hemos definido el conjunto de datos que representa el problema de asignación de los actores por tomas. Hemos definido una matriz de costo asociado a cada actor por cada toma.

A continuación vamos a definir las funciones auxiliares: valor, CI, crear_hijos y ramificación_y_poda. Donde la función **valor** calcula el valor de una solución parcial, **CI** calcula una cota inferior para la solución parcial, **crear_hijos** genera los hijos de un nodo para la asignación de actores por tomas, y, la función principal **ramificación_y_poda** ejecuta el algoritmo ramificación y poda.

En la ramificación y poda se van a generar nodos y ramificar el árbol de búsqueda a través de un bucle. En cada iteración se seleccionará el nodo más prometedor basándose de la cota inferior. Se generarán los jijos mediante la función "crear_hijos". Como es necesario también realizar una poda, se eliminarán los nodos cuya cota inferior es mayor o igual a la mejor solución encontrada hasta el momento.

Ahora lo siguiente sería: **Inicializamos la solución inicialmente con todo vacío**, y, nos vamos a ir guardando una **variable que llamamos NODOS**, los nodos que debemos explorar, los nodos vivos, con lo cual lo que hemos hecho es inicializar la solución.

Realizamos un **bucle iterativo**, lo que hacemos es ramificar, exploramos, **buscamos el nodo prometedor**. Hemos creado una **variable HIJOS** que va a guardar los hijos, porque lo que vamos a hacer posteriormente, es podarlos. **A continuación solamente añadiremos a NODOS a aquellos nodos que hayan pasado el filtro de la poda**.

Ultimo codigo ramificacion y poda heuristica

In [63]:
def CI(NODO, matriz_tomas):
    return sum(matriz_tomas[i][NODO[i]] for i in range(len(NODO)))

def generar_hijos_ordenados(NODO, DIMENSION, matriz_tomas):
    hijos = []
    for i in range(DIMENSION):
        if i not in NODO:
            hijo = {'s': NODO + (i,), 'ci': CI(NODO + (i,), matriz_tomas)}
            hijos.append(hijo)
    hijos.sort(key=lambda x: x['ci'])
    return hijos

def ramificacion_y_poda_heuristica(matriz_tomas):
    DIMENSION = len(matriz_tomas[0])
    mejor_solucion = tuple(i for i in range(DIMENSION))
    cota_sup = CI(mejor_solucion, matriz_tomas)

    nodos = [{'s': (), 'ci': CI((), matriz_tomas)}]
    iteracion = 0

    while nodos:
        iteracion += 1
        nodo_prometedor = min(nodos, key=lambda x: x['ci'])['s']
        hijos = generar_hijos_ordenados(nodo_prometedor, DIMENSION, matriz_tomas)

        for hijo in hijos:
            if len(hijo['s']) == DIMENSION:
                if hijo['ci'] < cota_sup:
                    cota_sup = hijo['ci']
                    mejor_solucion = hijo['s']
            elif hijo['ci'] < cota_sup:
                nodos.append(hijo)

        nodos = [x for x in nodos if x['s'] != nodo_prometedor]

    print("La solución final es:", mejor_solucion)

ramificacion_y_poda_heuristica(matriz_tomas)

La solución final es: (5, 1, 0, 2, 4, 6, 7, 3, 8, 9)


busqueda local heuristico

In [58]:
import numpy as np

# Función para calcular el gasto total de una asignación
def gasto_total(asignacion, matriz_tomas):
    num_dias, num_actores = asignacion.shape
    gasto_total = 0
    for i in range(num_dias):
        for j in range(num_actores):
            gasto_total += matriz_tomas[asignacion[i][j]][i]  # Sumar el costo de la toma asignada al actor en el día i
    return gasto_total

# Función para generar una solución inicial aleatoria
def generar_solucion_inicial(num_dias, num_actores, num_tomas_por_dia):
    # Generar una asignación aleatoria respetando las restricciones de número máximo de tomas por día
    asignacion = np.zeros((num_dias, num_actores), dtype=int)
    for i in range(num_dias):
        tomas_disponibles = np.arange(num_actores)
        np.random.shuffle(tomas_disponibles)
        asignacion[i, :num_tomas_por_dia] = tomas_disponibles[:num_tomas_por_dia]
    return asignacion

# Vecindad: intercambiar dos tomas entre días
def vecindad(asignacion):
    vecinos = []
    num_dias, num_actores = asignacion.shape
    for i in range(num_dias):
        for j in range(i + 1, num_dias):
            for k in range(num_actores):
                for l in range(num_actores):
                    vecino = asignacion.copy()
                    vecino[i, k], vecino[j, l] = vecino[j, l], vecino[i, k]
                    vecinos.append(vecino)
    return vecinos

# Búsqueda local heurística
def busqueda_local_heuristica(matriz_tomas, num_dias, num_actores, num_tomas_por_dia, max_iter):
    mejor_solucion = generar_solucion_inicial(num_dias, num_actores, num_tomas_por_dia)
    mejor_gasto = gasto_total(mejor_solucion, matriz_tomas)

    iteracion = 0
    mejora = True
    while mejora and iteracion < max_iter:
        mejora = False
        iteracion += 1
        for vecino in vecindad(mejor_solucion):
            gasto_vecino = gasto_total(vecino, matriz_tomas)
            if gasto_vecino < mejor_gasto:
                mejor_solucion = vecino
                mejor_gasto = gasto_vecino
                mejora = True

    return mejor_solucion, mejor_gasto

# Parámetros del problema
MAX_ITER = 1000
NUM_DIAS = 5  # Número de días de grabación
NUM_ACTORES = 10  # Número de actores
NUM_TOMAS_POR_DIA = 6  # Máximo número de tomas por día

# Ejecutar la búsqueda local heurística
mejor_solucion, gasto_mejor_solucion = busqueda_local_heuristica(matriz_tomas, NUM_DIAS, NUM_ACTORES, NUM_TOMAS_POR_DIA, MAX_ITER)

print("La mejor solución encontrada es:")
print(mejor_solucion)
print("Gasto total de la mejor solución:", gasto_mejor_solucion)


La mejor solución encontrada es:
[[2 1 0 4 1 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0]
 [5 6 8 5 5 2 6 7 3 8]
 [9 9 3 7 7 7 0 0 0 0]
 [4 8 3 9 0 8 0 0 0 0]]
Gasto total de la mejor solución: 23
